## Transformation (DataJob) Mover

**This tutorial shows how to copy transformations from one team/ data pool to another one, independent of the cluster.**



**To do so we first need to connect to the source data job.**

In [ ]:
from pycelonis import get_celonis

c_source = get_celonis("URL to the team from which you want to copy.", "Specify a valid API key for your source cloud team.")
source_data_pool = c_source.pools.find("Name or ID of the source data pool.")
source_data_job = source_data_pool.data_jobs.find("Name or ID of the source data job.")

**In the next step we connect to the target data job.**

In [ ]:
c_target = get_celonis("URL to cloud team where you want to copy to.", "Specify a valid API key for your target cloud team.")
target_data_pool = c_target.pools.find("Name or ID of the target data pool.")
target_data_job = target_data_pool.data_jobs.find("Name or ID of the target data job.")

**In this step we save all source global parameter IDs in a dictionary and overwrite them with the target global parameter ID if the parameter exists already in the target data pool.**

In [ ]:
global_vars = {}
for source_var in source_data_pool.variables:                                                          # loop through global parameters of source data pool
    global_vars.update({source_var.id: None})                                                          # save ID of source global parameter
    for target_var in target_data_pool.variables:                                                      # loop through the global parameters of target data pool
        if source_var.data['placeholder'].upper() == target_var.data['placeholder'].upper():           # if the placeholder of a source global parameter and a target global parameter match
            global_vars.update({source_var.id: target_var.id})                                         # match the saved ID of source global parameter wih the target global parameter ID
print("Global parameter configurations saved.")

**This section serves to create the target transformation, the related transformation parameters and to copy over the template settings.**

In [ ]:
for source_transformation in source_data_job.transformations.data:   # loop through source transformations
    
    if source_transformation.statement is None:                      # if the source transformation is empty, it will not be created
        continue
    
    # copy transformation from source to target data job:
    target_transformation = target_data_job.create_transformation(name=source_transformation.name, description=source_transformation.data['description'], statement=source_transformation.statement)
    print("Transformation: '" + target_transformation.name + "' created.")
    
    for source_local_var in source_transformation.variables:                                                                              # loop through the source transformation parameters
        if source_local_var['defaultSettings'] is not None:                                                                               # create the connection for the default value to the target global parameters
            if source_local_var['defaultSettings']['poolVariableId'] is not None:                                                                             
                target_id = global_vars.get(source_local_var['defaultSettings']['poolVariableId'])
                if target_id is None:
                    para = target_data_pool.create_pool_parameter(source_data_pool.variables.find(source_local_var['defaultSettings']['poolVariableId']))  # create the target global parameter if it does not exist yet
                    print("Pool Parameter '" + para.name + "' created.")
                    target_id = para.id
                    global_vars[source_local_var['defaultSettings']['poolVariableId']] = target_id
                source_local_var['defaultSettings']['poolVariableId'] = target_id
        if source_local_var['settings'] is not None:                                                                                      # create the connection for the value to the target global parameters 
            if source_local_var['settings']['poolVariableId'] is not None:
                target_id = global_vars.get(source_local_var['settings']['poolVariableId'])
                if target_id is None:
                    para = target_data_pool.create_pool_parameter(source_data_pool.variables.find(source_local_var['settings']['poolVariableId']))         # create the target global parameter if it does not exist yet
                    print("Pool Parameter '" + para.name + "' created.")
                    target_id = para.id
                    global_vars[source_local_var['defaultSettings']['poolVariableId']] = target_id
                source_local_var['settings']['poolVariableId'] = target_id

        target_transformation.create_transformation_parameter(source_local_var)                                                            # create the target transformation parameter
        print("Parameter '" + source_local_var['name'] + "' created.")
        
    if(source_transformation.data['template']):                                                                                            # copy template settings to target transformation
        target_transformation.to_template(source_transformation.data['protectionStatus'])

print("Congrats you copied the data job "+ source_data_job.name + "!")